In [1]:
neighbours = {
  'S': ['A','B'],
  'A': ['C','D','S'],
  'B': ['D','G','S'],
  'C': ['A','B','G','S'],
  'D': ['A','B'],
  'G': ['A','B','C','D']
}

S = 'S'
G = 'G'

In [2]:
class DoublyLinkedList:
    _items = []
    def __init__(self, items=None):
        if items is None:
            items = []
        self._items = items

    def is_empty(self):
        return len(self._items) == 0

    def length(self):
        return len(self._items)

    def head(self):
        if self.is_empty():
            return None
        return self._items[0]
    
    def tail(self):
        if self.is_empty():
            return None
        return DoublyLinkedList(self._items[1:])

    def add_to_end(self, value):
        self._items.append(value)

    def add_to_start(self, value):
        self._items.insert(0, value)

    def remove(self, value):
        self._items.remove(value)

    def __add__(self, other):
        result = DoublyLinkedList()
        result._items = self._items + other._items
        return result

    def __sub__(self, other):
        result = DoublyLinkedList()
        for item in self._items:
            if item not in other._items:
                result.add_to_end(item)
        return result

    def __contains__(self, value):
        return any(x[0] == value for x in self._items)

    def __str__(self):
        return str(self._items)

In [3]:
def MoveGen(node):
  return DoublyLinkedList(neighbours[node])

# Simple Search 1

In [4]:
def SimpleSearch1(S, G):
  open = DoublyLinkedList([S])
  while not open.is_empty():
    print(f"OPEN: {open}")
    N = open.head()
    open.remove(N)
    print(f"------------")
    if N == G:
      return N
    else:
      open = open + MoveGen(N)
  
  return None

In [5]:
SimpleSearch1(S, G)

OPEN: ['S']
------------
OPEN: ['A', 'B']
------------
OPEN: ['B', 'C', 'D', 'S']
------------
OPEN: ['C', 'D', 'S', 'D', 'G', 'S']
------------
OPEN: ['D', 'S', 'D', 'G', 'S', 'A', 'B', 'G', 'S']
------------
OPEN: ['S', 'D', 'G', 'S', 'A', 'B', 'G', 'S', 'A', 'B']
------------
OPEN: ['D', 'G', 'S', 'A', 'B', 'G', 'S', 'A', 'B', 'A', 'B']
------------
OPEN: ['G', 'S', 'A', 'B', 'G', 'S', 'A', 'B', 'A', 'B', 'A', 'B']
------------


'G'

# Simple Search 2

In [6]:
def SimpleSearch2(S, G):
  open = DoublyLinkedList([S])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    N = open.head()
    open.remove(N)
    closed.add_to_end(N)
    print(f"------------")
    if N == G:
      return N
    else:
      open = open + (MoveGen(N) - closed)
  
  return None

In [7]:
SimpleSearch2(S, G)

OPEN: 	['S']
CLOSED: []
------------
OPEN: 	['A', 'B']
CLOSED: ['S']
------------
OPEN: 	['B', 'C', 'D']
CLOSED: ['S', 'A']
------------
OPEN: 	['C', 'D', 'D', 'G']
CLOSED: ['S', 'A', 'B']
------------
OPEN: 	['D', 'D', 'G', 'G']
CLOSED: ['S', 'A', 'B', 'C']
------------
OPEN: 	['D', 'G', 'G']
CLOSED: ['S', 'A', 'B', 'C', 'D']
------------
OPEN: 	['G', 'G']
CLOSED: ['S', 'A', 'B', 'C', 'D', 'D']
------------


'G'

# DFS

In [8]:
def MakePairs(nodeList, parent):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    return DoublyLinkedList([(nodeList.head(), parent)]) + MakePairs(nodeList.tail(), parent)

def RemoveSeen(nodeList, open, closed):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    N = nodeList.head()
    if N in open or N in closed:
      return RemoveSeen(nodeList.tail(), open, closed)
    else:
      return DoublyLinkedList([N]) + RemoveSeen(nodeList.tail(), open, closed)

def SkipTo(parent, nodePairs):
  if parent == nodePairs.head()[0]:
    return nodePairs
  else:
    return SkipTo(parent, nodePairs.tail())

def ReconstructPath(nodePair, closed):
  node, parent = nodePair
  path = DoublyLinkedList([node])
  while parent is not None:
    path.add_to_start(parent)
    closed = SkipTo(parent, closed)
    _, parent = closed.head()
  return path

In [9]:
def DFS(S, G):
  open = DoublyLinkedList([(S, None)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _ = nodePair
    print(f"------------")
    if N == G:
      return ReconstructPath(nodePair, closed)
    else:
      closed.add_to_start(nodePair)
      neighbours = MoveGen(N)
      newNodes = RemoveSeen(neighbours, open, closed)
      newPairs = MakePairs(newNodes, N)
      open = newPairs + open.tail()
  return DoublyLinkedList([])

print(DFS(S, G))

OPEN: 	[('S', None)]
CLOSED: []
------------
OPEN: 	[('A', 'S'), ('B', 'S')]
CLOSED: [('S', None)]
------------
OPEN: 	[('C', 'A'), ('D', 'A'), ('B', 'S')]
CLOSED: [('A', 'S'), ('S', None)]
------------
OPEN: 	[('G', 'C'), ('D', 'A'), ('B', 'S')]
CLOSED: [('C', 'A'), ('A', 'S'), ('S', None)]
------------
['S', 'A', 'C', 'G']


# BFS

In [10]:
def BFS(S, G):
  open = DoublyLinkedList([(S, None)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _ = nodePair
    print(f"------------")
    if N == G:
      return ReconstructPath(nodePair, closed)
    else:
      closed.add_to_start(nodePair)
      neighbours = MoveGen(N)
      newNodes = RemoveSeen(neighbours, open, closed)
      newPairs = MakePairs(newNodes, N)
      open = open.tail() + newPairs
  return DoublyLinkedList([])

print(BFS(S, G))

OPEN: 	[('S', None)]
CLOSED: []
------------
OPEN: 	[('A', 'S'), ('B', 'S')]
CLOSED: [('S', None)]
------------
OPEN: 	[('B', 'S'), ('C', 'A'), ('D', 'A')]
CLOSED: [('A', 'S'), ('S', None)]
------------
OPEN: 	[('C', 'A'), ('D', 'A'), ('G', 'B')]
CLOSED: [('B', 'S'), ('A', 'S'), ('S', None)]
------------
OPEN: 	[('D', 'A'), ('G', 'B')]
CLOSED: [('C', 'A'), ('B', 'S'), ('A', 'S'), ('S', None)]
------------
OPEN: 	[('G', 'B')]
CLOSED: [('D', 'A'), ('C', 'A'), ('B', 'S'), ('A', 'S'), ('S', None)]
------------
['S', 'B', 'G']


# DFID-N

In [11]:
def MakePairs(nodeList, parent, depth):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    return DoublyLinkedList([(nodeList.head(), parent, depth)]) + MakePairs(nodeList.tail(), parent, depth)

def RemoveSeen(nodeList, open, closed):
  if nodeList.is_empty():
    return DoublyLinkedList([])
  else:
    N = nodeList.head()
    if N in open or N in closed:
      return RemoveSeen(nodeList.tail(), open, closed)
    else:
      return DoublyLinkedList([N]) + RemoveSeen(nodeList.tail(), open, closed)

def SkipTo(parent, nodePairs, depth):
  if parent == nodePairs.head()[0] and depth == nodePairs.head()[2]:
    return nodePairs
  else:
    return SkipTo(parent, nodePairs.tail(), depth)

def ReconstructPath(nodePair, closed):
  node, parent, depth = nodePair
  path = DoublyLinkedList([node])
  while parent is not None:
    path.add_to_start(parent)
    closed = SkipTo(parent, closed, depth - 1)
    _, parent, depth = closed.head()
  return path

In [14]:
def DB_DFS_N(S, G, depth_bound):
  count = 0
  open = DoublyLinkedList([(S, None, 0)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _, depth = nodePair
    
    if N == G:
      return (count, ReconstructPath(nodePair, closed))
    else:
      print(f"------------")
      closed.add_to_start(nodePair)

      if depth < depth_bound:
        neighbours = MoveGen(N)
        newNodes = RemoveSeen(neighbours, open, closed)
        newPairs = MakePairs(newNodes, N, depth+1)
        open = newPairs + open.tail()
        count = count + newPairs.length()
      else:
        open = open.tail()
  return (count, DoublyLinkedList([]))

def DFID_N(S, G):
  count = -1
  path = DoublyLinkedList([])
  depth_bound = 0

  while True:
    previous_count = count
    print(f"\n==========Depth {depth_bound}============")
    count, path = DB_DFS_N(S, G, depth_bound)
    depth_bound += 1
    if previous_count == count:
      break

  return path

print(DFID_N(S, G))


==========Depth 0============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------

==========Depth 1============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('B', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('B', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------

==========Depth 2============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('B', 'S', 1)]
CLOSED: [('S', None, 0)]
------------
OPEN: 	[('C', 'A', 2), ('D', 'A', 2), ('B', 'S', 1)]
CLOSED: [('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('D', 'A', 2), ('B', 'S', 1)]
CLOSED: [('C', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('B', 'S', 1)]
CLOSED: [('D', 'A', 2), ('C', 'A', 2), ('A', 'S', 1), ('S', None, 0)]
------------
OPEN: 	[('G', 'B', 2)]
CLOSED: [('B', 'S', 1), ('D', 'A', 2), ('C', 'A', 2), ('A', 'S', 1), ('S', None, 0)]

==========Depth 3============
OPEN: 	[('S', None, 0)]
CLOSED: []
------------
OPEN: 	[('A', 'S', 1), ('B', '

# DFID-C

## This Algorithm does not work with graph having cycles

In [13]:
def DB_DFS_C(S, G, depth_bound):
  count = 0
  open = DoublyLinkedList([(S, None, 0)])
  closed = DoublyLinkedList([])
  while not open.is_empty():
    print(f"OPEN: \t{open}")
    print(f"CLOSED: {closed}")
    nodePair = open.head()
    N, _, depth = nodePair
    
    if N == G:
      return (count, ReconstructPath(nodePair, closed))
    else:
      print(f"------------")
      closed.add_to_start(nodePair)

      if depth < depth_bound:
        neighbours = MoveGen(N)
        newNodes = RemoveSeen(neighbours, open, DoublyLinkedList([]))
        newPairs = MakePairs(newNodes, N, depth+1)
        open = newPairs + open.tail()
        count = count + newPairs.length()
      else:
        open = open.tail()
  return (count, DoublyLinkedList([]))

def DFID_C(S, G):
  count = -1
  path = DoublyLinkedList([])
  depth_bound = 0

  while True:
    previous_count = count
    print(f"==========Depth {depth_bound}============")
    count, path = DB_DFS_C(S, G, depth_bound)
    depth_bound += 1
    if previous_count == count:
      if (path.is_empty()):
        print("Empty path")
      if (previous_count == count):
        print("count equal")
      break

  return path

# run it if the graph on top does not have cycles
# print(DFID_C(S, G))